In [47]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re
import emoji
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error,r2_score,mean_squared_log_error
from sklearn.model_selection import GridSearchCV
from scipy.sparse import csr_matrix, hstack
from xgboost import XGBRegressor
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [7]:
df_price_dataset = pd.read_table(r'data\train.tsv')

In [8]:
display(df_price_dataset)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity
...,...,...,...,...,...,...,...,...
1482530,1482530,Free People Inspired Dress,2,Women/Dresses/Mid-Calf,Free People,20.0,1,"Lace, says size small but fits medium perfectl..."
1482531,1482531,Little mermaid handmade dress,2,Kids/Girls 2T-5T/Dresses,Disney,14.0,0,Little mermaid handmade dress never worn size 2t
1482532,1482532,21 day fix containers and eating plan,2,Sports & Outdoors/Exercise/Fitness accessories,NaN,12.0,0,"Used once or twice, still in great shape."
1482533,1482533,World markets lanterns,3,Home/Home Décor/Home Décor Accents,NaN,45.0,1,There is 2 of each one that you see! So 2 red ...


In [9]:
missing_values_count = df_price_dataset.isnull().sum()
#look at the missing value
print(missing_values_count)

train_id                  0
name                      0
item_condition_id         0
category_name          6327
brand_name           632682
price                     0
shipping                  0
item_description          6
dtype: int64


In [10]:
def process_category(data):
    for i in range(3):
        
        def get_part(x):
            
            if type(x) != str:
                return np.nan
        
            parts = x.split('/')
            
            if i >= len(parts):
                return np.nan
            else:
                return parts[i]

        field_name = 'category_' + str(i)
        
        data[field_name] = data['category_name'].apply(get_part)
    
    return data

In [11]:
process_category(df_price_dataset)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,category_0,category_1,category_2
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,Men,Tops,T-shirts
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces
...,...,...,...,...,...,...,...,...,...,...,...
1482530,1482530,Free People Inspired Dress,2,Women/Dresses/Mid-Calf,Free People,20.0,1,"Lace, says size small but fits medium perfectl...",Women,Dresses,Mid-Calf
1482531,1482531,Little mermaid handmade dress,2,Kids/Girls 2T-5T/Dresses,Disney,14.0,0,Little mermaid handmade dress never worn size 2t,Kids,Girls 2T-5T,Dresses
1482532,1482532,21 day fix containers and eating plan,2,Sports & Outdoors/Exercise/Fitness accessories,NaN,12.0,0,"Used once or twice, still in great shape.",Sports & Outdoors,Exercise,Fitness accessories
1482533,1482533,World markets lanterns,3,Home/Home Décor/Home Décor Accents,NaN,45.0,1,There is 2 of each one that you see! So 2 red ...,Home,Home Décor,Home Décor Accents


In [12]:
df_price_dataset.fillna({'brand_name': ' ', 'category_0': 'other', 'category_1': 'other', 'category_2': 'other'}, inplace = True)

In [13]:
df_price_dataset = df_price_dataset.drop(columns = ['category_name'], axis = 1)

In [14]:
missing_values_count = df_price_dataset.isnull().sum()
#look at the missing value
print(missing_values_count)

train_id             0
name                 0
item_condition_id    0
brand_name           0
price                0
shipping             0
item_description     6
category_0           0
category_1           0
category_2           0
dtype: int64


In [15]:
df_price_dataset = df_price_dataset.dropna()

In [16]:
display(df_price_dataset)

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,category_0,category_1,category_2
0,0,MLB Cincinnati Reds T Shirt Size XL,3,,10.0,1,No description yet,Men,Tops,T-shirts
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse
3,3,Leather Horse Statues,1,,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces
...,...,...,...,...,...,...,...,...,...,...
1482530,1482530,Free People Inspired Dress,2,Free People,20.0,1,"Lace, says size small but fits medium perfectl...",Women,Dresses,Mid-Calf
1482531,1482531,Little mermaid handmade dress,2,Disney,14.0,0,Little mermaid handmade dress never worn size 2t,Kids,Girls 2T-5T,Dresses
1482532,1482532,21 day fix containers and eating plan,2,,12.0,0,"Used once or twice, still in great shape.",Sports & Outdoors,Exercise,Fitness accessories
1482533,1482533,World markets lanterns,3,,45.0,1,There is 2 of each one that you see! So 2 red ...,Home,Home Décor,Home Décor Accents


In [17]:
def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [18]:
def clean_text(text):
    if not isinstance(text, str):
        return ''  # Return empty string for non-string inputs
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove single quotes, double quotes
    text = re.sub(r"[\'\"]", "", text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace('\\r', ' ')
    text = text.replace('\\"', ' ')
    text = text.replace('\\n', ' ')
    text = decontracted(text)
    # Remove emojis
    text = emoji.demojize(text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words and word not in list(string.punctuation)]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    cleaned_text = ' '.join(lemmatized_tokens)
    return cleaned_text

In [19]:
df_price_dataset['item_description'] = df_price_dataset['item_description'].apply(lambda x: clean_text(x))

In [20]:
display(df_price_dataset)

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,category_0,category_1,category_2
0,0,MLB Cincinnati Reds T Shirt Size XL,3,,10.0,1,description yet,Men,Tops,T-shirts
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,keyboard great condition work like came box po...,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse,1,Target,10.0,1,adorable top hint lace key hole back pale pink...,Women,Tops & Blouses,Blouse
3,3,Leather Horse Statues,1,,35.0,1,new tag leather horse retail rm stand foot hig...,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,,44.0,0,complete certificate authenticity,Women,Jewelry,Necklaces
...,...,...,...,...,...,...,...,...,...,...
1482530,1482530,Free People Inspired Dress,2,Free People,20.0,1,lace say size small fit medium perfectly never...,Women,Dresses,Mid-Calf
1482531,1482531,Little mermaid handmade dress,2,Disney,14.0,0,little mermaid handmade dress never worn size 2t,Kids,Girls 2T-5T,Dresses
1482532,1482532,21 day fix containers and eating plan,2,,12.0,0,used twice still great shape,Sports & Outdoors,Exercise,Fitness accessories
1482533,1482533,World markets lanterns,3,,45.0,1,2 one see 2 red 2 orange 2 big red orange one ...,Home,Home Décor,Home Décor Accents


In [79]:
def get_features(data):
    
    luxury_brands = ["MCM", "MCM Worldwide", "Louis Vuitton", "Burberry", "Burberry London", "Burberry Brit", "HERMES", "Tieks",
                     "Rolex", "Apple", "Gucci", "Valentino", "Valentino Garavani", "RED Valentino", "Cartier", "Christian Louboutin",
                     "Yves Saint Laurent", "Saint Laurent", "YSL Yves Saint Laurent", "Georgio Armani", "Armani Collezioni", "Emporio Armani"]
    data['is_luxurious'] = (data['brand_name'].isin(luxury_brands)).astype(np.int8)

    expensive_brands = ["Michael Kors", "Louis Vuitton", "Lululemon", "LuLaRoe", "Kendra Scott", "Tory Burch", "Apple", "Kate Spade",
                  "UGG Australia", "Coach", "Gucci", "Rae Dunn", "Tiffany & Co.", "Rock Revival", "Adidas", "Beats", "Burberry",
                  "Christian Louboutin", "David Yurman", "Ray-Ban", "Chanel"]
    data['is_expensive'] = (data['brand_name'].isin(expensive_brands)).astype(np.int8)
    return data

In [22]:
get_features(df_price_dataset)

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,category_0,category_1,category_2,is_luxurious,is_expensive
0,0,MLB Cincinnati Reds T Shirt Size XL,3,,10.0,1,description yet,Men,Tops,T-shirts,0,0
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,keyboard great condition work like came box po...,Electronics,Computers & Tablets,Components & Parts,0,0
2,2,AVA-VIV Blouse,1,Target,10.0,1,adorable top hint lace key hole back pale pink...,Women,Tops & Blouses,Blouse,0,0
3,3,Leather Horse Statues,1,,35.0,1,new tag leather horse retail rm stand foot hig...,Home,Home Décor,Home Décor Accents,0,0
4,4,24K GOLD plated rose,1,,44.0,0,complete certificate authenticity,Women,Jewelry,Necklaces,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1482530,1482530,Free People Inspired Dress,2,Free People,20.0,1,lace say size small fit medium perfectly never...,Women,Dresses,Mid-Calf,0,0
1482531,1482531,Little mermaid handmade dress,2,Disney,14.0,0,little mermaid handmade dress never worn size 2t,Kids,Girls 2T-5T,Dresses,0,0
1482532,1482532,21 day fix containers and eating plan,2,,12.0,0,used twice still great shape,Sports & Outdoors,Exercise,Fitness accessories,0,0
1482533,1482533,World markets lanterns,3,,45.0,1,2 one see 2 red 2 orange 2 big red orange one ...,Home,Home Décor,Home Décor Accents,0,0


In [23]:
df_price_dataset['name'] = df_price_dataset['name']+' '+df_price_dataset['brand_name']

In [24]:
df_price_dataset

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,category_0,category_1,category_2,is_luxurious,is_expensive
0,0,MLB Cincinnati Reds T Shirt Size XL,3,,10.0,1,description yet,Men,Tops,T-shirts,0,0
1,1,Razer BlackWidow Chroma Keyboard Razer,3,Razer,52.0,0,keyboard great condition work like came box po...,Electronics,Computers & Tablets,Components & Parts,0,0
2,2,AVA-VIV Blouse Target,1,Target,10.0,1,adorable top hint lace key hole back pale pink...,Women,Tops & Blouses,Blouse,0,0
3,3,Leather Horse Statues,1,,35.0,1,new tag leather horse retail rm stand foot hig...,Home,Home Décor,Home Décor Accents,0,0
4,4,24K GOLD plated rose,1,,44.0,0,complete certificate authenticity,Women,Jewelry,Necklaces,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1482530,1482530,Free People Inspired Dress Free People,2,Free People,20.0,1,lace say size small fit medium perfectly never...,Women,Dresses,Mid-Calf,0,0
1482531,1482531,Little mermaid handmade dress Disney,2,Disney,14.0,0,little mermaid handmade dress never worn size 2t,Kids,Girls 2T-5T,Dresses,0,0
1482532,1482532,21 day fix containers and eating plan,2,,12.0,0,used twice still great shape,Sports & Outdoors,Exercise,Fitness accessories,0,0
1482533,1482533,World markets lanterns,3,,45.0,1,2 one see 2 red 2 orange 2 big red orange one ...,Home,Home Décor,Home Décor Accents,0,0


In [25]:
df_price_dataset['name'] = df_price_dataset['name'].apply(lambda x: clean_text(x))

In [26]:
df_price_dataset.drop('brand_name', axis = 1,inplace=True)

In [27]:
display(df_price_dataset)

,train_id,name,item_condition_id,price,shipping,item_description,category_0,category_1,category_2,is_luxurious,is_expensive
0,0,mlb cincinnati red shirt size xl,3,10.0,1,description yet,Men,Tops,T-shirts,0,0
1,1,razer blackwidow chroma keyboard razer,3,52.0,0,keyboard great condition work like came box po...,Electronics,Computers & Tablets,Components & Parts,0,0
2,2,avaviv blouse target,1,10.0,1,adorable top hint lace key hole back pale pink...,Women,Tops & Blouses,Blouse,0,0
3,3,leather horse statue,1,35.0,1,new tag leather horse retail rm stand foot hig...,Home,Home Décor,Home Décor Accents,0,0
4,4,24k gold plated rose,1,44.0,0,complete certificate authenticity,Women,Jewelry,Necklaces,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1482530,1482530,free people inspired dress free people,2,20.0,1,lace say size small fit medium perfectly never...,Women,Dresses,Mid-Calf,0,0
1482531,1482531,little mermaid handmade dress disney,2,14.0,0,little mermaid handmade dress never worn size 2t,Kids,Girls 2T-5T,Dresses,0,0
1482532,1482532,21 day fix container eating plan,2,12.0,0,used twice still great shape,Sports & Outdoors,Exercise,Fitness accessories,0,0
1482533,1482533,world market lantern,3,45.0,1,2 one see 2 red 2 orange 2 big red orange one ...,Home,Home Décor,Home Décor Accents,0,0


In [55]:
df_price_dataset['shipping'].unique()

array([1, 0], dtype=int64)

In [28]:
data = df_price_dataset[['price', 'name', 'category_0', 'category_1',
       'category_2', 'shipping', 'item_condition_id', 'is_expensive', 'is_luxurious', 'item_description']]

In [29]:
y = data['price']
X = data.drop('price', axis = 1)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size = 0.5, random_state = 42)

In [30]:
def get_ohe(X_train,X_val, X_test, col_name):
    vect = CountVectorizer()
    tr_ohe = vect.fit_transform(X_train[col_name].values)
    val_ohe = vect.transform(X_val[col_name].values)
    te_ohe = vect.transform(X_test[col_name].values)
    return tr_ohe,val_ohe,te_ohe

In [31]:
def get_text_encodings(X_train,X_val,X_test, col_name, min_val, max_val):
    vect = TfidfVectorizer(min_df = 10, ngram_range = (min_val, max_val), max_features = 1000000)
    tr_text = vect.fit_transform(X_train[col_name].values)
    val_text = vect.transform(X_val[col_name].values)
    te_text = vect.transform(X_test[col_name].values)
    return tr_text,val_text, te_text

In [32]:
def generate_encodings(X_train,X_val,X_test):
    tr_ohe_category_0,val_ohe_category_0,te_ohe_category_0 = get_ohe(X_train,X_val, X_test, 'category_0')
    tr_ohe_category_1,val_ohe_category_1,te_ohe_category_1 = get_ohe(X_train,X_val, X_test, 'category_1')
    tr_ohe_category_2,val_ohe_category_2,te_ohe_category_2 = get_ohe(X_train,X_val,X_test, 'category_2')

    tr_trans = csr_matrix(pd.get_dummies(X_train[['shipping', 'item_condition_id', 'is_expensive', 'is_luxurious']], sparse=True).values)
    val_trans = csr_matrix(pd.get_dummies(X_val[['shipping', 'item_condition_id', 'is_expensive', 'is_luxurious']], sparse=True).values)
    te_trans = csr_matrix(pd.get_dummies(X_test[['shipping', 'item_condition_id', 'is_expensive', 'is_luxurious']], sparse=True).values)

    tr_name,val_name,te_name = get_text_encodings(X_train,X_val,X_test, 'name', 1, 1)
    tr_text, val_text,te_text = get_text_encodings(X_train,X_val,X_test, 'item_description', 1, 2)

    train_data = hstack((tr_ohe_category_0, tr_ohe_category_1, tr_ohe_category_2, tr_trans, \
                       tr_name, tr_text)).tocsr().astype('float32')
    
    val_data = hstack((val_ohe_category_0, val_ohe_category_1, val_ohe_category_2, val_trans, \
                       val_name, val_text)).tocsr().astype('float32')


    test_data = hstack((te_ohe_category_0, te_ohe_category_1, te_ohe_category_2, te_trans, \
                       te_name, te_text)).tocsr().astype('float32')

    return train_data, val_data,test_data

In [33]:
train_data, val_data,test_data = generate_encodings(X_train,X_val,X_test)
print(train_data.shape, y_train.shape)
print(val_data.shape, y_val.shape)
print(test_data.shape, y_test.shape)

(1186023, 276568) (1186023,)
(741264, 276568) (741264,)
(741265, 276568) (741265,)


In [36]:
tsvd = TruncatedSVD(n_components=1000)  # Adjust the number of components as needed
train_data_tsvd = tsvd.fit_transform(train_data)
val_data_tsvd = tsvd.fit_transform(val_data)
test_data_tsvd = tsvd.fit_transform(test_data)

In [37]:
def get_rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [48]:
xgb_model = XGBRegressor()
xgb_model.fit(train_data_tsvd, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [62]:
xgb_model.save_model('XGBoost_PricePrediction_Model.hdf5')

c:\Users\hemas\Documents\Applied_AI_and_ML_Courses\Foundations_Of_ML\labs\CSCN8010\venv\tensorflow_cpu\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:09:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [49]:
predictions_train = xgb_model.predict(train_data_tsvd)

In [58]:
predictions_train = np.maximum(0,predictions_train)

In [59]:
train_score = get_rmsle(y_train,predictions_train)
print('Train rmsle explained: ', train_score)


Train rmsle explained:  0.5783739193798634


In [172]:
predictions_val = xgb_model.predict(val_data_tsvd)

In [173]:
predictions_val = np.maximum(0,predictions_val)

In [175]:
val_score = get_rmsle(y_val,predictions_val)
print('Val rmsle explained: ', val_score)

Val rmsle explained:  0.7642290024003087


In [44]:
predictions_test = xgb_model.predict(test_data_tsvd)

In [60]:
predictions_test = np.maximum(0,predictions_test)

In [61]:
test_score = get_rmsle(y_test,predictions_test)
print('Train rmsle explained: ', test_score)


Train rmsle explained:  0.8010997335684786


In [66]:
loaded_model = XGBRegressor()
loaded_model.load_model('XGBoost_PricePrediction_Model.hdf5')

In [103]:
def get_ohe_single(sample, col_name):
    vect = CountVectorizer()
    if isinstance(sample[col_name], pd.Series):
        ohe = vect.fit_transform(sample[col_name].values.astype('U'))  # Convert to Unicode
    else:
        ohe = vect.fit_transform([sample[col_name]])
    return ohe

def get_text_encodings_single(sample, col_name, min_val, max_val):
    vect = TfidfVectorizer(min_df=min_val, ngram_range=(min_val, max_val), max_features=1000000)
    if isinstance(sample[col_name], pd.Series):
        encoding = vect.fit_transform(sample[col_name].values.astype('U'))
    else:
        encoding = vect.fit_transform([sample[col_name]])
    return encoding

def generate_encodings_single(sample):
    ohe_category_0 = get_ohe_single(sample, 'category_0')
    ohe_category_1 = get_ohe_single(sample, 'category_1')
    ohe_category_2 = get_ohe_single(sample, 'category_2')

    trans = csr_matrix(pd.get_dummies(sample[['shipping', 'item_condition_id','is_luxurious','is_expensive']], sparse=True).values)

    name_encoding = get_text_encodings_single(sample, 'name', 1, 1)
    text_encoding = get_text_encodings_single(sample, 'item_description', 1, 2)

    data = hstack((ohe_category_0, ohe_category_1, ohe_category_2, trans, name_encoding, text_encoding)).tocsr().astype('float32')

    return data

In [117]:
def preprocess(data):
    data = process_category(data)
    
    data['name'] = data['name'].apply(lambda x: clean_text(x))
    data['item_description'] = data['item_description'].apply(lambda x: clean_text(x))
    
    data = get_features(data)
    
    data.fillna({'brand_name': ' ', 'category_0': 'other', 'category_1': 'other', 'category_2': 'other'}, inplace=True)
    
    # Concat columns
    data['name'] = data['name'] + ' ' + data['brand_name']  # Assuming 'brand_name' is important for encoding
    
    return data

In [176]:
sample_data = [{
    'name': 'Levis Black Leggings, Womens. Size L.',
    'item_condition_id': 1,
    'category_name': 'Women/Athletic Apparel/Pants, Tights, Leggings',
    'brand_name': 'Levis',
    'shipping': 1,
    'item_description': 'Adorable gym wear from a well known brand. In great condition. Size L. Black and stretchable.'
}]
input_data = pd.DataFrame.from_dict(sample_data)


In [177]:
display(input_data)

,name,item_condition_id,category_name,brand_name,shipping,item_description
0,"Levis Black Leggings, Womens. Size L.",1,"Women/Athletic Apparel/Pants, Tights, Leggings",Levis,1,Adorable gym wear from a well known brand. In ...


In [178]:
def final_fun_1(input_data):
    
    print('Processing Data...')
    processed_data = preprocess(input_data)

    print('Generating Encodings...')
    encoded_data = generate_encodings_single(input_data)
    print(encoded_data.shape)
    target_num_features = 1000
    reshaped_arr = np.zeros((1, target_num_features))
    # Copy the existing data into the new array
    reshaped_arr[:, :encoded_data.shape[1]] = encoded_data.toarray()

    print(reshaped_arr.shape)

    print('Making Predictions...')

    pred = loaded_model.predict(reshaped_arr)
    print(pred)
    return pred

In [179]:
final_fun_1(input_data)

Processing Data...
Generating Encodings...
(1, 37)
(1, 1000)
Making Predictions...
[31.902088]


array([31.902088], dtype=float32)